In [7]:
import json

# Open the JSON file and load its content
with open('raw_classes.json', 'r') as file:
    data = json.load(file)

# for i in data["New item - 2"]["classes"]:
#     if i["class_section"] != "A1":
#         continue
#     print(i["descr"])

for i in data["New item - 2"]["classes"]:
    if i["class_section"] != "A1":
        continue
    print(f"{i["subject"]} {i["catalog_nbr"]} is called {i["descr"]}")

ENGEC 311 is called Introduction to Logic Design
ENGEC 327 is called Introduction to Software Engineering
ENGEC 330 is called Applied Algorithms for Engineers
ENGEC 401 is called Signals and Systems
ENGEC 402 is called Control Systems
ENGEC 410 is called Introduction to Electronics
ENGEC 412 is called Analog Electronics
ENGEC 413 is called Computer Organization
ENGEC 414 is called Introduction to Machine Learning
ENGEC 441 is called Introduction to Computer Networking
ENGEC 464 is called Senior Design Project 2
ENGEC 471 is called Physics of Semiconductor Devices
ENGEC 500 is called Special Topics in Electrical and Computer Engineering
ENGEC 503 is called Introduction to Learning from Data
ENGEC 504 is called Advanced Data Structures
ENGEC 512 is called Enterprise Client-Server Software Systems Design
ENGEC 513 is called Computer Architecture
ENGEC 520 is called Digital Image Processing and Communication
ENGEC 521 is called Cybersecurity
ENGEC 522 is called Computational Optical Imagin

subject: "ENGEC"
catalog_nbr: "311"
class_capacity: "82"
descr: "Introduction to Logic Design"
units: "4"
class_capacity: 82
meetings[days]: "TuTh"
meetings[start_time]: "09.00.00.000000"
meetings[end_time]: "10.45.00.000000"
meetings[facility_id]: "PHO 210"
meetings[instructor]: "Tali Moreshet"

In [31]:
import psycopg2
import json

# Connect to the PostgreSQL database
connection = psycopg2.connect(
    host="34.31.179.146",
    dbname="tutorialDB",
    user="postgres", 
    password="butlar"
)

cursor = connection.cursor()

# Create a table if it doesn't exist in the "python-test" schema
cursor.execute("""
    CREATE TABLE IF NOT EXISTS "coursework_info".courses (
        college VARCHAR(10),
        subject VARCHAR(10),
        catalog_nbr INT,
        prof_first TEXT,
        prof_last TEXT,
        building TEXT,
        room_num TEXT,
        class_capacity INT,
        descr TEXT,
        units TEXT,
        meeting_days TEXT,
        meeting_times TEXT,
        PRIMARY KEY (college, subject, catalog_nbr)
    );
""")

# cursor.execute("""
#     CREATE TABLE "coursework_info".professors (
#         professor_id SERIAL PRIMARY KEY, -- SQL creates a value for this automatically
#         first_name VARCHAR(50),
#         last_name VARCHAR(50)
#     );
# """)

# I don't think I need this

# cursor.execute("""
#     CREATE TABLE "coursework_info".professor_courses (
#         professor_id INT REFERENCES "coursework_info".professors(professor_id) ON DELETE CASCADE,
#         college VARCHAR(10),
#         subject VARCHAR(10),
#         catalog_nbr INT,
#         PRIMARY KEY (professor_id, college, subject, catalog_nbr),
#         FOREIGN KEY (college, subject, catalog_nbr) 
#             REFERENCES "coursework_info".courses(college, subject, catalog_nbr)
#     );
# """)


# Open and load the JSON file
with open('raw_classes.json', 'r') as file:
    data = json.load(file)

# Insert data into the table
for i in data["New item - 2"]["classes"]:
    if i["class_section"] != "A1":
        continue
    subject = i["subject"]
    if subject.startswith("ENG"):
        college = "ENG"
        subject = subject[3:]
    else:
        college = "NA"
        subject = "NA"
    catalog_nbr = i["catalog_nbr"]
    class_capacity = i["class_capacity"]
    descr = i["descr"]
    units = i["units"]
    professor_name = i["meetings"][0]["instructor"]
    prof_first, prof_last = professor_name.split(' ', 1)

    location = i["meetings"][0]["facility_id"]
    building, room_num = location.split(' ',1) if location != "TBA" else ("TBA", "TBA")
    meeting_days = i["meetings"][0]["days"]
    meeting_times = f"{i["meetings"][0]["start_time"][0:6].replace(".","")} - {i["meetings"][0]["end_time"][0:6].replace(".","")}"

    
    # Insert the course data into the "python-test" schema
    cursor.execute("""
        INSERT INTO "coursework_info".courses (college, subject, catalog_nbr, prof_first, prof_last, building, room_num, class_capacity, descr, units, meeting_days, meeting_times)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        ON CONFLICT (college, subject, catalog_nbr) DO NOTHING;
    """, (college, subject, catalog_nbr, prof_first, prof_last, building, room_num, class_capacity, descr, units, meeting_days, meeting_times))

    # cursor.execute("""
    #     INSERT INTO "coursework_info".professors (first_name, last_name)
    #     VALUES (%s, %s) RETURNING professor_id;
    # """, (prof_first, prof_last))
    # professor_id = cursor.fetchone()[0]

    # cursor.execute("""
    #     INSERT INTO "coursework_info".professor_courses (professor_id, college, subject, catalog_nbr)
    #     VALUES (%s, %s, %s, %s);
    # """, (professor_id, college, subject, catalog_nbr)
    # )

# Commit the transaction
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()